<a href="https://colab.research.google.com/github/manavsrivastav14/NLP-miniproject/blob/main/loading_model_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [3]:
from fastai import *
import torch
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb


In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
! gdown --id 1oRWNkYp3lU8Hq8kdmkddXEprObr177CD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1oRWNkYp3lU8Hq8kdmkddXEprObr177CD
To: /content/pa-train.csv
100% 10.2M/10.2M [00:00<00:00, 43.2MB/s]


In [8]:
df_train = pd.read_csv(path/'/content/pa-train.csv', header=None)

In [9]:
! gdown --id 1PvbCH67SmgdWLDp0m1kEz2C8PWLNwf_r
df_valid = pd.read_csv(path/'/content/pa-valid.csv', header=None)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1PvbCH67SmgdWLDp0m1kEz2C8PWLNwf_r
To: /content/pa-valid.csv
100% 1.27M/1.27M [00:00<00:00, 122MB/s]


In [10]:
! gdown --id 186o_Jh8ufmsd_uSiSkzTZMGNPWURcHQh
df_test = pd.read_csv('pa-test.csv', header=None)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=186o_Jh8ufmsd_uSiSkzTZMGNPWURcHQh
To: /content/pa-test.csv
100% 1.32M/1.32M [00:00<00:00, 133MB/s]


In [11]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [12]:
label_cols = [0]

In [13]:
class PanjabiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('panjabi_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [14]:
! gdown --id 16y0GO7fglAMRl3mB-tUa2kY1kRjzyM6i

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16y0GO7fglAMRl3mB-tUa2kY1kRjzyM6i
To: /content/panjabi_lm.model
100% 4.22M/4.22M [00:00<00:00, 74.3MB/s]


In [15]:
sp = spm.SentencePieceProcessor()
sp.Load('/content/panjabi_lm.model')
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [16]:
panjabi_vocab = Vocab(itos)

In [17]:
panjabi_tok = PanjabiTokenizer('pb')

In [18]:
tokenizer = Tokenizer(tok_func=PanjabiTokenizer, lang='pb')

In [19]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [20]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=panjabi_vocab, label_cols=label_cols)


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [21]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False)

In [22]:
learn.load('/content/drive/MyDrive/NLP/model', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (2496 items)
x: LMTextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹ

In [23]:
learn.predict('ਜੋ ਉਹਨਾਂ ਦੇ ਰੱਬਾਂ ਨੂੰ ਪ੍ਰਸਤੁਤ ਕਰਦੇ',n_words=10)

'ਜੋ ਉਹਨਾਂ ਦੇ ਰੱਬਾਂ ਨੂੰ ਪ੍ਰਸਤੁਤ ਕਰਦੇ ▁ਹਨ ▁ਅਤੇ ▁ਕੁਦਰਤ ▁ਦਾ ▁ਅੰਦਰ ▁ਜਾਣ ੇਗਾ ▁ਇਸ ▁ਮੇਲੇ ▁ਨੂੰ'

In [25]:
learn.save_encoder('/content/drive/MyDrive/NLP/fine_tuned_encoder')

In [26]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=panjabi_vocab, label_cols=label_cols, bs=64)


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [27]:

data_clas.show_batch()

text,target
"▁x x bo s ▁60 ▁ਸਾਲ ▁ਦਾ ▁ਪਾਪ ▁ਧੋਣ ▁ਦੀ ▁ਕੋਸ਼ਿਸ਼ ▁ਕਰ ▁ਰਹੀ ▁ਹੈ ▁ਕਾਂਗਰਸ <unk> ▁p ub lish <unk> ▁d ate : t u e , ▁0 9 <unk> ▁a pr ▁ 2019 ▁0 6 : 00 <unk> <unk> ▁( i <unk> st ) ▁ਚੋਣ ▁ਜ਼ਾਬ ਤੇ ▁ਕਾਰਨ ▁ਸਰਕਾਰ ▁ਦੇ ▁ਕੰਮਕਾਜ ▁ਰੁਕ ▁ਜਾਂਦੇ ▁ਹਨ । ▁ਦੂਜਾ ▁ਮੇਰੀ ▁ਕੋਸ਼ਿਸ਼ ▁ਹੁੰਦੀ ▁ਹੈ ▁ਕਿ ▁ਵੱਧ ▁ਤੋਂ ▁ਵੱਧ ▁ਲੋਕਾਂ ▁ਨੂੰ ▁ਮਿਲਾ । ▁ਇ",politics
"▁x x bo s ▁ਵਿਜ਼ ਨ ▁ਸਬਕ ਾ ▁ਸਾਥ - ਸਬ ਕਾ ▁ਵਿਕਾਸ ▁ 2019 -20 <unk> ▁p ub lish <unk> ▁d ate : s un , ▁31 <unk> ▁ 2019 ▁ 08 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਪਿਛਲੇ ▁ਇਕ ▁ਦਹਾਕੇ ▁' ਚ ▁ਭਾਰਤੀ ▁ਅਰਥਚਾਰਾ ▁ਕੌਮਾਂਤਰੀ ▁ਵਿਕਾਸ ▁' ਚ ▁ਸਭ ▁ਤੋਂ ▁ਵੱਡੇ ▁ਯੋਗਦਾਨ ▁ਕਰ ਤਾਵਾਂ ▁' ਚੋਂ ▁ਇਕ ▁ਰਿਹਾ ▁ਹੈ , ▁ਜੋ ▁ਆਰਥਿਕ ▁ਸਰਗਰਮੀਆਂ ▁' ਚ",politics
"▁x x bo s ▁ਸੁਪਰ ▁ਓਵਰ ▁' ਚ ▁ਜਿੱਤਿਆ ▁ਮੁ ਬੰ ਈ , ▁ਪੁੱਜਾ ▁ਪਲੇ ਆ ਫ ▁' ਚ <unk> ▁f ri , ▁ 03 ▁ <unk> <unk> ▁ 2019 ▁12 : 0 6 <unk> <unk> ▁( i <unk> st ) ▁| ▁ਹਾਈ ਲਾਈਟ ▁ਮਨੀਸ਼ ▁ਪਾਂਡੇ ▁ਨੇ ▁ਖੇਡੀ ▁71 ▁ਦੌੜਾਂ ▁ਦੀ ▁ਪਾਰੀ ▁ਵੀ ▁ਨਾ ▁ਜਿ ਤਾ ▁ਸਕੀ ▁ਹੈਦਰਾਬਾਦ ▁ਨੂੰ ▁ਹੈਦਰਾਬਾਦ ▁ਨੂੰ ▁ ਵਾਰਨਰ ▁ਦੀ ▁ਰੜਕ ੇਗੀ ▁ਘਾਟ ▁ਸੁਪਰ ▁ਓਵਰ ▁'",sports
"▁x x bo s ▁ਲੋਕਤੰਤਰ ▁ਦੇ ▁ਪਹਿਰੇਦਾਰ <unk> ▁p ub lish <unk> ▁d ate : m on , ▁15 <unk> ▁a pr ▁ 2019 ▁12 : 29 <unk> <unk> ▁( i <unk> st ) ▁ਚੋਣਾਂ ▁ਆਉਂਦੇ ▁ਹੀ ▁ਸਾਰੇ ▁ਦਲ , ▁ਵੋਟ , ▁ਚੋਣ ▁ਨਿਸ਼ਾਨ , ▁ਉਮੀਦਵਾਰ ▁ਤੇ ▁ਦੇਸ਼ ▁ਦੀਆਂ ▁ਵੱਡੀਆਂ - ਵੱਡੀਆਂ ▁ਸਿਆਸੀ ▁ਪਾਰਟੀਆਂ ▁ਦੀ ▁ਗੱਲ ▁ਕਰਦੇ ▁ਹਨ । ▁ਪਰ ▁ਇਸ ▁ਸ਼ੋਰ ▁ਵਿਚਕਾਰ ▁ਅਸੀਂ ▁ਉਸ ▁ਦੀ ▁ਭੂਮਿਕਾ ▁ਭੁੱਲ ▁ਜਾਂਦੇ ▁ਹਾਂ",politics
"▁x x bo s ▁ਬ੍ਰਾਜ਼ੀਲ ▁ਦੀ ▁ਮਾਰ ਤਾ ▁ਨੇ ▁ਸਿਰਜਿਆ ▁ਰਿਕਾਰਡ <unk> ▁p ub lish <unk> ▁d ate : f ri , ▁28 <unk> ▁j un ▁ 2019 ▁10 : 17 <unk> <unk> ▁( i <unk> st ) ▁ਫਰਾਂਸ ▁ਵਿਚ ▁ਖੇਡੇ ▁ਜਾ ▁ਰਹੇ ▁ਵਿਸ਼ਵ ▁ਫੁੱਟਬਾਲ ▁ਕੱਪ ▁' ਚ ▁ਬ੍ਰਾਜ਼ੀਲੀ ▁ਮਹਿਲਾ ▁ਟੀਮ ▁ਦੀ ▁ਕਪਤਾਨ ▁ਮਾਰ ਤਾ ▁ਨੇ ▁17 ਵਾਂ ▁ਗੋਲ ▁ਸਕੋਰ ▁ਕਰ ▁ਕੇ ▁ਇਤਿਹਾਸ ▁ਸਿਰਜਿਆ ▁ਹੈ । ▁ਬ੍ਰਾਜ਼ੀਲ ▁ਦਾ ▁ਫੁੱਟਬਾਲ ▁ਖੇਡ",sports


In [28]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)

In [29]:
learn.load_encoder('/content/drive/MyDrive/NLP/fine_tuned_encoder')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2496 items)
x: TextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹਨ । ▁

In [30]:
learn.loss_func.func

CrossEntropyLoss()

In [31]:
mcc = MatthewsCorreff()

In [32]:
learn.metrics = [accuracy, mcc]

In [33]:
learn.fit_one_cycle(2, 1e-2, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,accuracy,matthews_correff,time
0,0.351644,0.193595,0.967949,0.957154,22:09
1,0.222031,0.133141,0.958333,0.944402,22:28


Better model found at epoch 0 with accuracy value: 0.9679487347602844.


In [40]:
# ! pip install fastbook
# from fastbook import load_learner
# new_classifier =  load_learner('/content/drive/MyDrive/NLP', 'gen_model')
new_classifier =  load_learner('/content/drive/MyDrive/NLP','fine_tuned_encoder.pth')
# new_classifier.load_state_dict('/content/drive/MyDrive/Colab/text_classifier_encoder')
dir(new_classifier)
#--
# new_classifier = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)
# new_classifier.load_encoder('/content/drive/MyDrive/Colab/text_classifier_encoder')
# dir(new_classifier)


KeyError: ignored

In [35]:
import fastai
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

KeyboardInterrupt: ignored

In [ ]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])